<a href="https://colab.research.google.com/github/4zmyyy/Belajarpyhton/blob/main/Python_Learning_Day_4_Sankey_Flow_Tokenization_Trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import plotly.graph_objects as go

# Percobaan untuk membuat posisi encoding
def get_positional_encoding(max_seq_len, d_model):
    pos = np.arange(max_seq_len)[:, np.newaxis]
    dim = np.arange(d_model)[np.newaxis, :]
    angle = pos / np.power(10000, (2 * (dim // 2)) / d_model)
    angle[:, 0::2] = np.sin(angle[:, 0::2])
    angle[:, 1::2] = np.cos(angle[:, 1::2])
    return angle

# Implementasi Self-Attention dan cara kerjanya
class SelfAttention(tf.keras.layers.Layer):
    def __init__(self, d_model):
        super(SelfAttention, self).__init__()
        self.d_model = d_model
        self.query = tf.keras.layers.Dense(d_model)
        self.key = tf.keras.layers.Dense(d_model)
        self.value = tf.keras.layers.Dense(d_model)

    def call(self, x):
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)
        attention_scores = tf.matmul(Q, K, transpose_b=True) / tf.sqrt(tf.cast(self.d_model, tf.float32))
        attention_weights = tf.nn.softmax(attention_scores, axis=-1)
        return tf.matmul(attention_weights, V), attention_weights

# Contoh penggunaan dan aplikasinya
max_seq_len = 5
d_model = 64
positional_encoding = get_positional_encoding(max_seq_len, d_model)
x = tf.constant(positional_encoding, dtype=tf.float32)

# Membuat layer self-attention
attention_layer = SelfAttention(d_model)
output, attention_weights = attention_layer(x)

# Visualisasi dengan model Sankey Diagram
fig = go.Figure()
source = []
target = []
value = []

for i in range(max_seq_len):
    for j in range(max_seq_len):
        source.append(i)
        target.append(j)
        value.append(attention_weights[i, j].numpy())

fig.add_trace(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=[f"Token {i}" for i in range(max_seq_len)]
    ),
    link=dict(
        source=source,
        target=target,
        value=value
    )
))

fig.update_layout(title_text="Sankey Diagram of Attention Weights", font_size=12)
fig.show()